In [3]:
import pandas as pd
label = pd.read_csv('label.csv')
edge = pd.read_csv('edge.csv')

label


,id,label
0,u17461978,human
1,u1297437077403885568,bot
2,u17685258,human
3,u15750898,human
4,u1659167666,bot
...,...,...
11821,u452754350,bot
11822,u850435801687183362,bot
11823,u2188795745,bot
11824,u940687680,bot


In [4]:
import json
from pprint import pprint
# Specify the path to your JSON file
json_file = 'TwiBot-20_sample.json'


In [8]:
from datetime import datetime
import pandas as pd
import numpy as np


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from kerastuner.tuners import RandomSearch, Hyperband


/var/folders/pc/81jrlndn7nsbmwkc6xg371gm0000gn/T/ipykernel_36027/1433536498.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, Hyperband


In [9]:
import pandas as pd
data = pd.read_json('TwiBot-20_sample.json')


data
df_exploded = data.explode('tweet')

df_exploded

,ID,profile,tweet,neighbor,domain
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",RT @CarnivalCruise: 🎉 Are you ready to see wha...,None,"[Politics, Bussiness, Entertainment]"
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",Who has time for receipts? Not me. @epson rece...,None,"[Politics, Bussiness, Entertainment]"
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",Steady wants to encourage you to invest in you...,None,"[Politics, Bussiness, Entertainment]"
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...","Good one, @rishid. But let’s see if y'all can ...",None,"[Politics, Bussiness, Entertainment]"
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",#lsunationalchamps\n,None,"[Politics, Bussiness, Entertainment]"
...,...,...,...,...,...
99,56321478,"{'id': '56321478 ', 'id_str': '56321478 ', 'na...",RT @RobRuadh: Meanwhile in Melbourne...\n\n“Oh...,"{'following': ['743560108509007872', '85498982...",[Politics]
99,56321478,"{'id': '56321478 ', 'id_str': '56321478 ', 'na...",RT @RaymondArroyo: Laura @IngrahamAngle and @M...,"{'following': ['743560108509007872', '85498982...",[Politics]
99,56321478,"{'id': '56321478 ', 'id_str': '56321478 ', 'na...",@JuliansRum they need to change their outfits ...,"{'following': ['743560108509007872', '85498982...",[Politics]
99,56321478,"{'id': '56321478 ', 'id_str': '56321478 ', 'na...",@ScoopskiPotatoo @dantevos95 they are all like...,"{'following': ['743560108509007872', '85498982...",[Politics]


In [10]:
label['id'] = label['id'].str[1:]

label

,id,label
0,17461978,human
1,1297437077403885568,bot
2,17685258,human
3,15750898,human
4,1659167666,bot
...,...,...
11821,452754350,bot
11822,850435801687183362,bot
11823,2188795745,bot
11824,940687680,bot


In [11]:
df_exploded['ID'] = df_exploded['ID'].astype('Int64')
label['id'] = label['id'].astype('Int64')
result = df_exploded.merge(label, left_on='ID', right_on='id', how='left')


df_cleaned = result.dropna(subset=['label'])

df_cleaned['type'] = df_cleaned['label'].apply(lambda x: 0 if x == 'human' else 1)

df_cleaned = df_cleaned[['ID','tweet','type']]

df_cleaned = df_cleaned.rename(columns={'tweet':'Text'})
df_cleaned = df_cleaned.dropna(subset=['Text'])

df_cleaned

/var/folders/pc/81jrlndn7nsbmwkc6xg371gm0000gn/T/ipykernel_36027/3664512218.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['type'] = df_cleaned['label'].apply(lambda x: 0 if x == 'human' else 1)


,ID,Text,type
0,17461978,RT @CarnivalCruise: 🎉 Are you ready to see wha...,0
1,17461978,Who has time for receipts? Not me. @epson rece...,0
2,17461978,Steady wants to encourage you to invest in you...,0
3,17461978,"Good one, @rishid. But let’s see if y'all can ...",0
4,17461978,#lsunationalchamps\n,0
...,...,...,...
17368,56321478,RT @RobRuadh: Meanwhile in Melbourne...\n\n“Oh...,1
17369,56321478,RT @RaymondArroyo: Laura @IngrahamAngle and @M...,1
17370,56321478,@JuliansRum they need to change their outfits ...,1
17371,56321478,@ScoopskiPotatoo @dantevos95 they are all like...,1


In [13]:
import re
import pandas as pd
import numpy as np
import torch
from transformers import MobileBertTokenizer, MobileBertModel

# Load MobileBERT tokenizer
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

# Shorten text to 295 characters
def truncate_text(text):
    return text[:127]

# Remove non-alphanumeric characters
df_cleaned['Text'] = df_cleaned['Text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

# Convert text to lowercase
df_cleaned['Text'] = df_cleaned['Text'].str.lower()

# Truncate text to 512 characters

df_cleaned['Text'] = df_cleaned['Text'].apply(truncate_text)

# Tokenize twitter data
inputs = tokenizer(list(df_cleaned['Text']), padding=True, truncation=True, return_tensors="pt")

# Load MobileBERT model
model = MobileBertModel.from_pretrained('google/mobilebert-uncased')

# Pass the inputs through the MobileBERT model
with torch.no_grad():
    outputs = model(**inputs)

# Extract the embeddings from the last hidden state
embeddings = outputs.last_hidden_state


/Users/suhelkeswani/Desktop/CS4641Project-main/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
from sklearn.preprocessing import StandardScaler
# Average the embeddings 
embeddings_mean = embeddings.mean(dim=1)

# Convert embeddings tensor to numpy array
embeddings_array = embeddings_mean.numpy()

# Normalize embeddings
scaler = StandardScaler()
embeddings_normalized = scaler.fit_transform(embeddings_array)

# Create a DataFrame with the normalized embeddings
df_transformed = pd.DataFrame(embeddings_normalized)

# Concatenate the new DataFrame with the original DataFrame
final_df = pd.concat([df_cleaned.reset_index(drop=True), df_transformed.reset_index(drop=True)], axis=1)

print(final_df)

             ID                                               Text  type  \
0      17461978  rt carnivalcruise  are you ready to see what o...     0   
1      17461978  who has time for receipts not me epson receipt...     0   
2      17461978  steady wants to encourage you to invest in you...     0   
3      17461978  good one rishid but lets see if yall can do be...     0   
4      17461978                                lsunationalchamps\n     0   
...         ...                                                ...   ...   
17156  56321478  rt robruadh meanwhile in melbourne\n\noh dan p...     1   
17157  56321478  rt raymondarroyo laura ingrahamangle and marco...     1   
17158  56321478  juliansrum they need to change their outfits a...     1   
17159  56321478  scoopskipotatoo dantevos95 they are all like i...     1   
17160  56321478  rt redbarnpatriot cenkuygur realdonaldtrump th...     1   

              0         1         2         3         4         5         6  \
0      0

In [34]:
tweet_train = final_df.iloc[:,3:]
tweet_test = final_df.iloc[:,2]

In [33]:
## KNN CLASSIFIER ##

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def knn_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    knn_model = KNeighborsClassifier()
    knn_param_grid = {'n_neighbors': np.arange(1, 15)}
    knn_grid_search = GridSearchCV(knn_model, knn_param_grid, cv=5)
    knn_grid_search.fit(X_train, y_train)
    
    knn_optimal = knn_grid_search.best_params_['n_neighbors']
    knn_optimized_model = KNeighborsClassifier(n_neighbors=knn_optimal)
    knn_optimized_model.fit(X_train, y_train)
    predictions = knn_optimized_model.predict(X_test)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='binary')
    recall = recall_score(y_test, predictions, average='binary')
    f1 = f1_score(y_test, predictions, average='binary')
    
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")
    
    return knn_optimized_model

knn_model(tweet_train, tweet_test)

Accuracy: 0.785
Precision: 0.100
Recall: 0.003
F1 Score: 0.005


KNeighborsClassifier(n_neighbors=14)

In [46]:
## RANDOM FOREST ##

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

def rf_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    rf_model = RandomForestClassifier()
    rf_param_grid = {'n_estimators': [10, 20, 30], 'max_depth': [1, 2], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}    
    rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, scoring='accuracy')
    rf_grid_search.fit(X_train, y_train)
    rf_optimized_model = rf_grid_search.best_estimator_
    rf_optimized_model.fit(X_train, y_train)
    predictions = rf_optimized_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")

    return rf_optimized_model


rf_model(tweet_train, tweet_test)

Accuracy: 0.790
Precision: 0.624
Recall: 0.790
F1 Score: 0.697


/Users/suhelkeswani/Desktop/CS4641Project-main/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForestClassifier(max_depth=1, n_estimators=10)

In [74]:
## NEURAL NET ##

from tensorflow.keras import layers, models, optimizers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import shutil
import os
tf.keras.backend.clear_session()

tuner_directory = 'my_dir/ml_4641'
if os.path.exists(tuner_directory):
    shutil.rmtree(tuner_directory)

# Initialize training and test data
X = tweet_train
y = tweet_test

def twitter_nn_class_model(hp):
    # Input Layer
    inputs = layers.Input(shape=(X.shape[1],))  
    
    # Hidden Layer with relu activation 
    x = layers.Dense(units=hp.Int('units', min_value=2, max_value=16, step=2), activation='relu')(inputs)

    # Output Layer with sigmoid activation for binary classification
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    # Create model
    model = models.Model(inputs, outputs)

    # Adam Optimizer 
    model.compile(optimizer=optimizers.Adam(hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])),
                  loss='binary_crossentropy')
    return model

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Create hyperparameter tuning for model
tuner = RandomSearch(
    twitter_nn_class_model,
    objective='val_loss',
    max_trials=24, 
    executions_per_trial=1, 
    directory='my_dir',
    project_name='ml_4641'
)

# Run hyperparameter search
tuner.search(X_train, y_train, epochs=50, validation_split=0.3)

# Get best performing model
best_model = tuner.get_best_models(num_models=1)[0]

# Get best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
print("Best Hyperparameters:")
print("Units:", best_hp.get('units'))
print("Learning Rate:", best_hp.get('learning_rate'))

# Predict using the best model
y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")


Trial 22 Complete [00h 00m 10s]
val_loss: 0.5153102874755859

Best val_loss So Far: 0.5139055848121643
Total elapsed time: 00h 03m 52s
Best Hyperparameters:
Units: 14
Learning Rate: 0.001
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step
Accuracy: 0.795
Precision: 0.632
Recall: 0.795
F1 Score: 0.704


/Users/suhelkeswani/Desktop/CS4641Project-main/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/suhelkeswani/Desktop/CS4641Project-main/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
